In [6]:
# Import the requests library for handling HTTP requests (Not used in the provided code snippet).
import requests

# Import the pdfplumber library to work with PDF files.
import pdfplumber

# Import the pandas library for data manipulation and analysis.
import pandas as pd

# Import the regular expression library for text pattern matching.
import re

# Import namedtuple from the collections library to create tuple subclasses with named fields.
from collections import namedtuple

# Define a variable 'invoice' holding the filename of the PDF.
invoice = 'xFY2312 09 TIGER OCT 12.pdf'

# Function to extract all text from a PDF file.
def extract_text_from_pdf(invoice):
    # Open the PDF file specified by 'invoice' using pdfplumber.
    with pdfplumber.open(invoice) as pdf:
        # Initialize an empty string to collect all text from the PDF.
        all_text = ''
        # Iterate through each page in the PDF.
        for page in pdf.pages:
            # Extract text from the current page.
            text = page.extract_text()
            # Append the extracted text to 'all_text', adding a newline for separation.
            all_text += text + '\n'
    # Return the concatenated text from all pages.
    return all_text

# Call the function with the 'invoice' filename and store the result in 'all_text'.
all_text = extract_text_from_pdf(invoice)

# Output the extracted text.
all_text

"Tl,GERNatura IG as.He.\nP.O. BOX 702437\nTulsa, OK 74170-2437 INVOICE NUMBER: 9239036\n918 491-6998\n918 491-6659 Fax INVOICE DATE: 10/12/2023\nUTILITY PROMPT PAY\nPAGE: 1\nSPECIAL INSTRUCTIONS:\nattn: Cynthia Cooper Donald.Garland@dla.mil\nFederal Aviation Administration, MMAC Paul.Rossi@dla.mil\nMike Monroney Aeronautical Center Timothy.Weatherspoon@dla.mil\nAAQ-811 Facilities and Aviation Safety cynthia.cooper@faa.gov\nPO Box 25082 MPB Room 380 kevin.coker@faa.gov\nOklahoma City, OK 73125\nCONTRACT # SPE604-23-D-7516\n405-954-2601 Line Item #0007-2\nCUSTOMER I.D. ONG Account Number PAYMENT TERMS\nFAA MMAC 210357426 1326106 021 NET 30 DAYS\nSALES REP I.D. Service Address SHIP DATE DUE DATE\n6500 South MacArthur Blvd 11/11/2023\nQUANTITY DESCRIPTION UNIT PRICE EXTENSION\n1980 DTH Sep-23 NATURAL GAS DELIVERIES $ 2.8077 $ 5,559.25\n318 DTH Sep-23 NATURAL GAS DELIVERIES $ 2.8802 $ 915.90\n2298\n6,475.15\nKEVIN L Digitally signed by KEVIN L\nCOKER\nCOKER Date: 2023.10.17 11:41:32 -05'00'

In [7]:
lines = all_text.split('\n')

formatted_text = '\n'.join(lines)

print(formatted_text)

Tl,GERNatura IG as.He.
P.O. BOX 702437
Tulsa, OK 74170-2437 INVOICE NUMBER: 9239036
918 491-6998
918 491-6659 Fax INVOICE DATE: 10/12/2023
UTILITY PROMPT PAY
PAGE: 1
SPECIAL INSTRUCTIONS:
attn: Cynthia Cooper Donald.Garland@dla.mil
Federal Aviation Administration, MMAC Paul.Rossi@dla.mil
Mike Monroney Aeronautical Center Timothy.Weatherspoon@dla.mil
AAQ-811 Facilities and Aviation Safety cynthia.cooper@faa.gov
PO Box 25082 MPB Room 380 kevin.coker@faa.gov
Oklahoma City, OK 73125
CONTRACT # SPE604-23-D-7516
405-954-2601 Line Item #0007-2
CUSTOMER I.D. ONG Account Number PAYMENT TERMS
FAA MMAC 210357426 1326106 021 NET 30 DAYS
SALES REP I.D. Service Address SHIP DATE DUE DATE
6500 South MacArthur Blvd 11/11/2023
QUANTITY DESCRIPTION UNIT PRICE EXTENSION
1980 DTH Sep-23 NATURAL GAS DELIVERIES $ 2.8077 $ 5,559.25
318 DTH Sep-23 NATURAL GAS DELIVERIES $ 2.8802 $ 915.90
2298
6,475.15
KEVIN L Digitally signed by KEVIN L
COKER
COKER Date: 2023.10.17 11:41:32 -05'00'
SUBTOTAL $ 6,475.15
SALES/U

In [8]:
import re
import calendar

# Function to convert date formats to MM/DD/YY
def convert_date_format(date_str):
    return re.sub(r'(\d{2})/(\d{2})/(\d{4})', lambda m: f"{m.group(1)}/{m.group(2)}/{m.group(3)[-2:]}", date_str)

def extract_data(all_text, current_invoice_date):
    # Regex search patterns
    dates = re.findall(r'\d{1,2}/\d{1,2}/\d{4}', all_text)
    invoice_number_match = re.search(r'INVOICE NUMBER: (\d+)', all_text)
    invoice_number = invoice_number_match.group(1) if invoice_number_match else "Unknown Invoice Number"
    
    if len(dates) >= 2:
        start_date, end_date = dates[0], dates[-1]
    else:
        start_date = end_date = "Unknown Date"
    
    usage_search = re.search(r'Usage (\d+)', all_text)
    usage = f"{int(usage_search.group(1)):,.2f}" if usage_search else "0.00"

    total_due_match = re.search(r'TOTAL DUE \$ ([\d,]+.\d{2})', all_text)
    total_due = total_due_match.group(1) if total_due_match else "Unknown Total Due"
    
    signature_date_match = re.search(r'Date: (\d{4}.\d{2}.\d{2})', all_text)
    if signature_date_match:
        # Parse the date from the format YYYY.MM.DD
        parsed_date = datetime.strptime(signature_date_match.group(1), '%Y.%m.%d')
        # Format it to dd/mm/yy
        signature_date = parsed_date.strftime('%m/%d/%y')
    else:
        signature_date = "Unknown Signature Date"
    
    
    end_date_formatted = convert_date_format(end_date) if 'Unknown Date' not in end_date else end_date
    
    # Extract month and year from invoice_date and format it as YY-MM
    invoice_date_match = re.search(r'INVOICE DATE:\s*(\d{1,2}[-/]\d{1,2}[-/]\d{4})', all_text, re.IGNORECASE)
    # if invoice_date_match:
    #     invoice_date = invoice_date_match.group(1)
    #     invoice_day, invoice_month, invoice_year_suffix = invoice_date.split('/')
    #     formatted_invoice_date = f"{invoice_year_suffix[-2:]}-{invoice_month}"
    formatted_invoice_date = current_invoice_date

    # Extracting the month and year from end_date_formatted for the start date
    month, day, year_suffix = end_date_formatted.split('/') if 'Unknown Date' not in end_date_formatted else (None, None, None)
    if month and year_suffix:
        month = int(month)
        year = int(f"20{year_suffix}")
        month_name = calendar.month_name[month][:3]
        _, last_day = calendar.monthrange(year, month)
        start_date_formatted = f"{month}/{last_day}/{year_suffix}"
    else:
        month_name = "Unknown Month"
        start_date_formatted = "Unknown Start Date"
    
    # Set payment terms based on the days of the month of the end date
    if month and year:
        _, days_in_month = calendar.monthrange(year, month)
        payment_terms = f"{days_in_month}"
    else:
        payment_terms = "Unknown Payment Terms"

    # Extracting specific quantities and prices
    first_quantity_search = re.search(r'Plus 10%\s*(\d+)', all_text)
    first_quantity = int(first_quantity_search.group(1)) if first_quantity_search else 0
    
    first_unit_price_search = re.search(r'Contract \$ \$\s*(\d+\.\d+)', all_text)
    first_unit_price = float(first_unit_price_search.group(1)) if first_unit_price_search else 0.0
    
    # Calculating first extension as product of first quantity and first unit price
    first_extension = first_quantity * first_unit_price
    
    second_quantity = int(float(usage.replace(",", ""))) - first_quantity if usage != "0.00" else 0
    
    second_unit_price_search = re.search(r'Differential \$ (\d+\.\d+)', all_text)
    second_unit_price = float(second_unit_price_search.group(1)) if second_unit_price_search else 0.0
    
    # Extracting second extension directly from text
    second_extension_search = re.search(r'Cashout \$ (\d+,?\d+\.\d+)', all_text)
    second_extension = second_extension_search.group(1) if second_extension_search else "0.00"
    
    extracted_data = f"{invoice_number}\tMMAC \t{formatted_invoice_date}\t{month_name}\t{end_date_formatted}\t{start_date_formatted}\t{payment_terms}\t{usage}\t{first_quantity}\t${first_unit_price:.4f}\t${first_extension:.2f}\t{second_quantity}\t${second_unit_price:.4f}\t${second_extension}\t${total_due}\t{signature_date}\t${total_due}"
    return extracted_data

In [9]:
import pandas as pd
from datetime import datetime, timedelta

# Define the starting date for the invoices
start_date = datetime(2023, 12, 1)

# Initialize an empty list to store invoice names
invoice_names = []

column_names = [
    "Invoice #", "BuildingGroup", "Period", "Month", "DateFrom", "DateTo",
    "Days", "Delivered DTH", "DTH-Nomination", "Nom $/DTH", "Nom $$", "DTH-O/U",
    "O/U $/DTH", "O/U $$", "Total", "DateAccepted", "AmountAccepted"
]
data_frame = pd.DataFrame(columns=column_names)

# Open the file and read lines
with open('invoices.txt', 'r') as file:
    invoice_names = [line.strip() for line in file.readlines()]

for invoice in invoice_names: 
    all_text = extract_text_from_pdf(invoice)
    # Format the current date as a string if needed by extract_data
    current_invoice_date = start_date.strftime("%y-%m")
    extracted_data = extract_data(all_text, current_invoice_date)  # Make sure this function returns data formatted as expected
    # Split the extracted_data string into a list of values
    data_values = extracted_data.split('\t')
    # Create a DataFrame for the current data
    temp_df = pd.DataFrame([data_values], columns=column_names)
    # Use concat to add the new row to the main DataFrame
    data_frame = pd.concat([data_frame, temp_df], ignore_index=True)
    # Increment the month by one
    start_date += timedelta(days=(calendar.monthrange(start_date.year, start_date.month)[1]))

data_frame

,Invoice #,BuildingGroup,Period,Month,DateFrom,DateTo,Days,Delivered DTH,DTH-Nomination,Nom $/DTH,Nom $$,DTH-O/U,O/U $/DTH,O/U $$,Total,DateAccepted,AmountAccepted
0,9239036,MMAC,23-12,Sep,9/1/2023,9/30/2023,30,"2,298.00",1980,$2.8077,$5559.25,318,$2.8802,$915.90,"$6,475.15",10/17/23,"$6,475.15"
1,10239036,MMAC,24-01,Oct,10/1/2023,10/31/2023,31,"5,602.00",4400,$2.8777,$12661.88,1202,$2.8243,"$3,394.81","$16,056.69",11/16/23,"$16,056.69"
2,11239036,MMAC,24-02,Nov,11/1/2023,11/30/2023,30,"8,776.00",9350,$3.2777,$30646.49,-574,$0.0000,$0.00,"$28,765.10",Unknown Signature Date,"$28,765.10"


In [10]:
file_path = "exported_data.xlsx"

# Export the DataFrame to an Excel file
data_frame.to_excel(file_path, index=False, engine='openpyxl')